In [ ]:
import pandas as pd
from sklearn.utils import resample
from sklearn import preprocessing
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Flatten, Reshape
from tensorflow.keras.models import Model

# Load the dataset
data = pd.read_csv('/content/loan_data (1).csv')
pd.set_option('display.max_columns', None)
data.drop(columns=['SK_ID_CURR'], inplace=True)
data.drop(columns=[col for col in data.columns if col.startswith('FLAG_DOCUMENT_')], inplace=True)
data.drop(columns=[col for col in data.columns if col.startswith('EXT_SOURCE_')], inplace=True)

# Fill null values depending on the column type
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = data[column].fillna(data[column].mode()[0])
    elif data[column].dtype == 'float64':
        data[column] = data[column].fillna(data[column].median())
    elif data[column].dtype == 'int64':
        data[column] = data[column].fillna(round(data[column].median()))

# Label encoding for categorical columns
le = preprocessing.LabelEncoder()
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = le.fit_transform(data[column])

# Separating the minority and majority classes
data_target_0 = data[data.TARGET == 0]
data_target_1 = data[data.TARGET == 1]

# Upsample the minority class
data_target_1_upsampled = resample(data_target_1, replace=True, n_samples=data_target_0.shape[0], random_state=123)

# Combine the majority class with upsampled minority class
data_upsampled = pd.concat([data_target_1_upsampled, data_target_0])

# Reset the index of the upsampled dataframe
data_upsampled.reset_index(drop=True, inplace=True)

# Display the first few rows of the dataframe
data_upsampled.head()

from sklearn.model_selection import train_test_split
# Assuming 'data_upsampled' is already defined and preprocessed

# Prepare data
X_data = data_upsampled.drop('TARGET', axis=1)
y_data = data_upsampled['TARGET']

# Normalize your data (if not already done)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(data_upsampled.drop('TARGET', axis=1))
y = data_upsampled['TARGET']

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


# Prediction Transformer
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


def build_ft_transformer_model(num_features, scale=10):
    inputs = Input(shape=(num_features,))
    x = inputs
    x = Reshape((1, -1))(x)  # Reshape for MultiHeadAttention
    x = Dense(64, activation='relu')(x)
    x = LayerNormalization()(x)
    attn_output = MultiHeadAttention(num_heads=2, key_dim=32)(x, x)
    attn_output = Dropout(0.1)(attn_output)
    x = LayerNormalization()(attn_output + x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy', 'mse', 'mae'])
    return model

# Initialize model with Fourier features
newmodel1 = build_ft_transformer_model(X_train.shape[1], scale=10)

# Setup callbacks for saving the best model and early stopping
checkpoint = ModelCheckpoint('best_model_inference.keras', monitor='val_loss', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Model training
history = newmodel1.fit(X_train, y_train,
                       epochs=1,
                       batch_size=32,
                       validation_data=(X_test, y_test),
                       callbacks=[checkpoint, early_stopping])


# Load the saved model
# model_path = '/content/best_model_inference.keras'
# tmodel1 = load_model(model_path)

import numpy as np
# Function to generate counterfactuals
def find_counterfactual(model, input_features, target_prediction, learning_rate=0.01, max_iterations=1000):
    input_features = tf.Variable(input_features, dtype=tf.float32)
    for iteration in range(max_iterations):
        with tf.GradientTape() as tape:
            tape.watch(input_features)
            prediction = model(input_features[tf.newaxis, ...])
            loss = tf.math.abs(prediction - target_prediction)

        gradients = tape.gradient(loss, input_features)

        if tf.reduce_all(tf.math.abs(prediction - target_prediction) < 0.01):
            break

        input_features.assign_sub(learning_rate * gradients)

    return input_features.numpy()

non_approved_indices = np.where(y_train.ravel() == 0)[0][:3]

def plot_feature_changes(X_sample, counterfactual, title):
    feature_changes = counterfactual - X_sample
    top_features_indices = np.argsort(np.abs(feature_changes))[-5:]
    plt.figure(figsize=(10, 6))
    plt.bar(range(5), feature_changes[top_features_indices], color='skyblue')
    plt.xlabel('Feature Index')
    plt.ylabel('Change in Feature Value')
    plt.title(title)
    plt.xticks(range(5), list(X_data.columns[list(top_features_indices)]))
    plt.xticks(rotation=45)
    plt.show()

for idx, example_index in enumerate(non_approved_indices):
    X_sample = X_train[example_index]
    counterfactual = find_counterfactual(newmodel1, X_sample, target_prediction=1.0)
    plot_title = f'Top 5 Feature Changes for Training Example {example_index} Counterfactual'
    plot_feature_changes(X_sample, counterfactual, plot_title)
# counterfactual = find_counterfactual(newmodel1, X_sample, target_prediction=1.0)

  423/11597 [>.............................] - ETA: 1:01 - loss: 0.6754 - accuracy: 0.5935 - mse: 0.2405 - mae: 0.4647

In [ ]:
# Predict probabilities using the trained model
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)  # Apply the default 0.5 threshold for binary classification

# Evaluation
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)  # Note: ROC-AUC is calculated from probabilities, not binary predictions
cm = confusion_matrix(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Output the evaluation results
print(f"F1 Score: {f1:.2f}")
print(f"ROC-AUC Score: {roc_auc:.2f}")
print(f"Confusion Matrix:\n{cm}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(
    X, y, data_upsampled.index, test_size=0.2, random_state=0)

# Normalize your data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predicting probabilities using the trained model
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Add predictions back to the dataframe
data_upsampled_test = data_upsampled.loc[indices_test]
data_upsampled_test['PREDICTION'] = y_pred

# Define mappings for CODE_GENDER and NAME_EDUCATION_TYPE
gender_mapping = {0: 'F', 1: 'M', 2: 'Other'}
education_mapping = {0: 'Academic degree', 1: 'Secondary / secondary special', 2: 'Incomplete higher', 3: 'Lower secondary', 4: 'Higher education'}

# Function to calculate fairness metrics
def calculate_fairness_metrics(data, sensitive_columns, mappings, outcome_column='TARGET', prediction_column='PREDICTION'):
    metrics = {}
    for sensitive_column in sensitive_columns:
        groups = data[sensitive_column].unique()
        group_metrics = {}
        for group in groups:
            group_name = mappings[sensitive_column].get(group, f'Unknown Group {group}')
            group_data = data[data[sensitive_column] == group]
            # Confusion matrix elements
            tn, fp, fn, tp = confusion_matrix(group_data[outcome_column], group_data[prediction_column]).ravel()

            # Calculating metrics
            tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
            fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
            positive_rate = (tp + fp) / group_data.shape[0]

            group_metrics[group_name] = {'TPR': tpr, 'FPR': fpr, 'Positive Rate': positive_rate}
        metrics[sensitive_column] = group_metrics
    return metrics

# Call the function with the desired columns and mappings
sensitive_columns = ['CODE_GENDER', 'NAME_EDUCATION_TYPE']
mappings = {'CODE_GENDER': gender_mapping, 'NAME_EDUCATION_TYPE': education_mapping}
fairness_metrics = calculate_fairness_metrics(data_upsampled_test, sensitive_columns, mappings)

# Output the results
for column, values in fairness_metrics.items():
    print(f"Fairness metrics for {column}:")
    for group_name, metrics in values.items():
        print(f"  {group_name}:")
        for metric, value in metrics.items():
            print(f"    {metric}: {value:.2f}")

# Function to calculate additional fairness metrics
def calculate_additional_fairness_metrics(data, sensitive_columns, mappings, outcome_column='TARGET', prediction_column='PREDICTION'):
    metrics = {}
    for sensitive_column in sensitive_columns:
        groups = data[sensitive_column].unique()
        group_metrics = {}
        for group in groups:
            group_name = mappings[sensitive_column].get(group, f'Unknown Group {group}')
            group_data = data[data[sensitive_column] == group]
            # Accuracy
            accuracy = accuracy_score(group_data[outcome_column], group_data[prediction_column])
            # Confusion matrix elements
            tn, fp, fn, tp = confusion_matrix(group_data[outcome_column], group_data[prediction_column]).ravel()

            # Calculating metrics
            total = tn + fp + fn + tp
            pr = (tp + fp) / total if total > 0 else 0
            tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
            disparity = pr / max(tpr, 1e-8)  # to avoid division by zero

            # Theil index
            p = pr
            q = 1 - pr
            t_index = 0 if p == 0 else p * np.log(p / q)

            group_metrics[group_name] = {'Accuracy': accuracy, 'Disparate Impact': disparity, 'Theil Index': t_index}
        metrics[sensitive_column] = group_metrics
    return metrics

# Call the function with the desired columns and mappings
sensitive_columns = ['CODE_GENDER', 'NAME_EDUCATION_TYPE']
mappings = {'CODE_GENDER': gender_mapping, 'NAME_EDUCATION_TYPE': education_mapping}
additional_fairness_metrics = calculate_additional_fairness_metrics(data_upsampled_test, sensitive_columns, mappings)

# Output the results
for column, values in additional_fairness_metrics.items():
    print(f"Additional fairness metrics for {column}:")
    for group_name, metrics in values.items():
        print(f"  {group_name}:")
        for metric, value in metrics.items():
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_curve, f1_score, roc_auc_score, confusion_matrix, mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Flatten, Reshape
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Load and preprocess data

X = data_upsampled.drop('TARGET', axis=1)
y = data_upsampled['TARGET']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data before applying SMOTE to avoid data leakage
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)

# Apply SMOTE only to training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
            print(f"    {metric}: {value:.2f}")

In [ ]:
class FourierFeatures(tf.keras.layers.Layer):
    def __init__(self, num_features, scale=10, **kwargs):
        super(FourierFeatures, self).__init__(**kwargs)
        self.num_features = num_features
        self.scale = scale

    def call(self, inputs):
        x = inputs
        for _ in range(self.num_features):
            x = tf.concat([x, tf.sin(x * self.scale), tf.cos(x * self.scale)], axis=-1)
        return x

def build_ft_transformer_model(num_features, fourier_features=True, scale=10):
    inputs = Input(shape=(num_features,))
    x = inputs
    if fourier_features:
        x = Dense(64, activation='relu')(inputs)
        x = LayerNormalization()(x)
        # Apply Fourier transformations directly in the Dense layer
        x = tf.concat([x, tf.sin(x * scale), tf.cos(x * scale)], axis=-1)
    x = Reshape((1, -1))(x)  # Reshape for MultiHeadAttention
    x = Dense(64, activation='relu')(x)
    x = LayerNormalization()(x)
    attn_output = MultiHeadAttention(num_heads=2, key_dim=32)(x, x)
    attn_output = Dropout(0.1)(attn_output)
    x = LayerNormalization()(attn_output + x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy', 'mse', 'mae'])
    return model

# Initialize model with Fourier features
newmodel1 = build_ft_transformer_model(X_train_smote.shape[1], fourier_features=True, scale=10)

# Setup callbacks for saving the best model and early stopping
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Model training
history = newmodel1.fit(X_train_smote, y_train_smote,
                       epochs=50,
                       batch_size=32,
                       validation_data=(X_test, y_test),
                       callbacks=[checkpoint, early_stopping])




# Plotting the training and validation loss
plt.figure(figsize=(14, 4))

# Plot training & validation loss values
plt.subplot(1, 3, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

# Plot training & validation MSE values
plt.subplot(1, 3, 2)
plt.plot(history.history['mse'], label='Train MSE')
plt.plot(history.history['val_mse'], label='Test MSE')
plt.title('Model MSE')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend()

# Plot training & validation MAE values
plt.subplot(1, 3, 3)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Test MAE')
plt.title('Model MAE')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve

def adjust_thresholds(y_true, y_pred_prob):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    # Maximize Youden's index (J = TPR - FPR)
    j_index = tpr - fpr
    optimal_idx = np.argmax(j_index)
    optimal_threshold = thresholds[optimal_idx]
    return optimal_threshold

# Predictions and threshold adjustment
y_pred_prob = newmodel1.predict(X_test)
optimal_threshold = adjust_thresholds(y_test, y_pred_prob)  # Ensure adjust_thresholds is correctly defined
y_pred_adjusted = (y_pred_prob > optimal_threshold).astype(int)

# Evaluation
f1 = f1_score(y_test, y_pred_adjusted)
roc_auc = roc_auc_score(y_test, y_pred_prob)
cm = confusion_matrix(y_test, y_pred_adjusted)
mse = mean_squared_error(y_test, y_pred_adjusted)
mae = mean_absolute_error(y_test, y_pred_adjusted)

print(f"F1 Score: {f1:.2f}")
print(f"ROC-AUC Score: {roc_auc:.2f}")
print(f"Confusion Matrix:\n{cm}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")

X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(
    X, y, data_upsampled.index, test_size=0.2, random_state=0)

# Normalize your data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Predicting probabilities using the trained model
y_pred_prob = newmodel1.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Add predictions back to the dataframe
data_upsampled_test = data_upsampled.loc[indices_test]
data_upsampled_test['PREDICTION'] = y_pred
fairness_metrics = calculate_fairness_metrics(data_upsampled_test, sensitive_columns, mappings)

# Output the results
for column, values in fairness_metrics.items():
    print(f"Fairness metrics for {column}:")
    for group_name, metrics in values.items():
        print(f"  {group_name}:")
        for metric, value in metrics.items():
            print(f"    {metric}: {value:.2f}")

additional_fairness_metrics = calculate_additional_fairness_metrics(data_upsampled_test, sensitive_columns, mappings)

# Output the results
for column, values in additional_fairness_metrics.items():
    print(f"Additional fairness metrics for {column}:")
    for group_name, metrics in values.items():
        print(f"  {group_name}:")
        for metric, value in metrics.items():
            print(f"    {metric}: {value:.2f}")